In [1]:
import numpy as np
import pandas as pd

# Constants

In [95]:
TRAINING_DATA_FILE = 'SpamData\\01_Processing\\train_data.txt'
TEST_DATA_FILE = 'SpamData\\01_Processing\\test_data.txt'

TOKEN_SPAM_PROB_FILE = 'SpamData\\03_Testing\\prob_spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData\\03_Testing\\prob_non_spam.txt'
TOKEN_ALL_PROB_FILE = 'SpamData\\03_Testing\\prob_all.txt'

TEST_FEATURE_MATIX = 'SpamData\\03_Testing\\test-features.txt'
TEST_TARGET_FILE = 'SpamData\\03_Testing\\test-target.txt'
SPAM_CAT = 1
HAM_CAT =0
VOCAB_SIZE = 2500

# Read and Load Features from .txt files into Numpy Array

In [9]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE,
                               delimiter=' ',
                               dtype=int)

In [117]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE,
                               delimiter=' ',
                               dtype=int)

# How to create empty dataframe

In [11]:
column_names = ['DOC_ID']+['CATAGORY']+list(range(0,VOCAB_SIZE))

In [12]:
len(column_names)

2502

In [13]:
index_names = np.unique(sparse_train_data[:,0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [14]:
full_train_data = pd.DataFrame(index=index_names,columns=column_names)

In [15]:
full_train_data.head()

,DOC_ID,CATAGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
full_train_data.fillna(value=0,inplace=True)

In [17]:
full_train_data.head()

,DOC_ID,CATAGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Create a full matrix from a Sparase matrix

In [29]:
def make_full_matrix(sparse_matrix,nr_words,doc_idx=0,word_idx=1,
                    cat_idx=2,freq_idx=3):
    
    """"
    From a full matrix from a sparse matrix. returns pandas dataframe
    Key words:
    sparse_matrix --numpy array
    nr_words --size of the vocabulary. Total number of tokens.
    doc_idx -- position of the document id in the parse matrix.
    word_idx --position of the word id in the sparse matrix.
    cat_idx--position of the label 
    
    """
    column_names = ['DOC_ID']+['CATAGORY']+list(range(0,VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:,0])
    full_matrix = pd.DataFrame(index=index_names,columns=column_names)
    full_matrix.fillna(value=0,inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr,'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr,'CATAGORY'] = label
        full_matrix.at[doc_nr,word_id] = occurrence
        
    full_matrix.set_index('DOC_ID',inplace=True)
    
    return full_matrix
    
    

In [30]:
%%time

full_train_data = make_full_matrix(sparse_train_data,VOCAB_SIZE)

Wall time: 14.6 s


In [121]:
sparse_train_data.shape

(258303, 4)

# Training the Naive Bayes Model

## Calcualting the Probabilty of Spam

In [43]:
counts = full_train_data['CATAGORY'].value_counts()
prob_spam = counts[1]/ (counts[0]+counts[1])
print('Probability of email beaing spam is: ',prob_spam)
#counts

Probability of email beaing spam is:  0.31133250311332505


## Total Number of words / Token

In [44]:
full_train_features = full_train_data.loc[:,full_train_data.columns != 'CATAGORY']

In [45]:
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,13,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
email_lengths = full_train_features.sum(axis=1)
email_lengths.shape

(4015,)

In [51]:
email_lengths[:5]

DOC_ID
0     87
1     53
2     40
3    181
4     43
dtype: int64

In [52]:
total_wc = email_lengths.sum()
total_wc

430952

# Number of tokens in spam & ham emails

In [56]:
spam_train_features_cat = full_train_data[full_train_data['CATAGORY'] == 1]

spam_train_features = spam_train_features_cat.loc[:,spam_train_features_cat.columns != 'CATAGORY']

spam_train_features.shape

(1250, 2500)

In [57]:
ham_train_features_cat = full_train_data[full_train_data['CATAGORY'] == 0]

ham_train_features = ham_train_features_cat.loc[:,ham_train_features_cat.columns != 'CATAGORY']

ham_train_features.shape

(2765, 2500)

In [59]:
spam_email_lengths = spam_train_features.sum(axis=1)
spam_email_lengths[:5]

DOC_ID
0     87
1     53
2     40
3    181
4     43
dtype: int64

In [60]:
ham_email_lengths = ham_train_features.sum(axis=1)
ham_email_lengths[:5]

DOC_ID
1896     82
1898    112
1899     66
1900     49
1901     34
dtype: int64

In [61]:
spam_total_wc = spam_email_lengths.sum()
spam_total_wc

178668

In [62]:
ham_total_wc = ham_email_lengths.sum()
ham_total_wc

252284

# other method

In [66]:
spam_lengths = email_lengths[full_train_data.CATAGORY==1]
ham_lengths = email_lengths[full_train_data.CATAGORY==0]


(1250,)

In [68]:
ham_wc = ham_lengths.sum()
spam_wc = spam_lengths.sum()

In [69]:
ham_wc +spam_wc - total_wc

0

In [74]:
print('average no of words in spam emails {:.0f} '.
      format(spam_wc/spam_lengths.shape[0]))
print('average no of words in non spam emails {:.0f} '.
      format(ham_wc/ham_lengths.shape[0]))

average no of words in spam emails 143 
average no of words in non spam emails 91 


# Summing the Tokens Occuring in Spam

In [75]:
full_train_features.shape

(4015, 2500)

In [76]:
train_spam_tokens = full_train_features.loc[full_train_data.CATAGORY ==1]
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,13,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [77]:
train_spam_tokens.tail()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1885,1,0,0,2,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1887,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1889,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1890,2,0,0,0,1,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1895,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [78]:
## Laplas smothing

summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1

summed_spam_tokens.shape

(2500,)

In [79]:
summed_spam_tokens.tail()

2495    19
2496     1
2497    18
2498    20
2499    16
dtype: int64

In [80]:
train_ham_tokens = full_train_features.loc[full_train_data.CATAGORY ==0]
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1
summed_ham_tokens.shape

(2500,)

## P(Token|Spam)-Probability that a Token Occurs given the Email is spam

In [85]:
prob_tokens_spam = summed_spam_tokens/(spam_wc+1)
prob_tokens_ham = summed_ham_tokens/(ham_wc +1)

In [94]:
type(prob_tokens_ham)

pandas.core.series.Series

# P(Token)-Probability that Token Occurs

In [87]:
prob_tokens_all = full_train_features.sum(axis=0)/total_wc

In [92]:
np.savetxt(TOKEN_SPAM_PROB_FILE,prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE,prob_tokens_ham)
np.savetxt(TOKEN_ALL_PROB_FILE,prob_tokens_all)

# Prepar test data

In [122]:
sparse_test_data[0:5]

array([[8, 2, 1, 1],
       [8, 3, 1, 4],
       [8, 4, 1, 2],
       [8, 5, 1, 1],
       [8, 6, 1, 2]])

In [97]:
%%time

full_test_data = make_full_matrix(sparse_test_data,nr_words=VOCAB_SIZE)

Wall time: 2min 55s


In [110]:
X_test = full_test_data.loc[:,full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATAGORY

In [105]:
np.savetxt(TEST_TARGET_FILE,y_test)
np.savetxt(TEST_FEATURE_MATIX,X_test)

In [104]:
X_test.shape

(5739, 2501)

In [123]:
np.savetxt(TEST_FEATURE_MATIX,X_test)

In [124]:
X_test.tail()

,CATAGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
5783.0,0.0,2.0,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5786.0,0.0,5.0,5.0,2.0,2.0,1.0,2.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5788.0,0.0,NaN,4.0,NaN,2.0,4.0,3.0,3.0,1.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5792.0,0.0,2.0,2.0,NaN,1.0,NaN,NaN,2.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5793.0,0.0,1.0,9.0,1.0,NaN,NaN,1.0,2.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
